In [1]:
from svgpathtools import svg2paths, wsvg, Path

In [2]:
def glob(file, suffix):
    '''Combine SVG paths into one compound path.'''
    paths, attributes, svg_attributes = svg2paths(file, return_svg_attributes=True)
    attribs = attributes[0]['fill']
    print(attribs)
    compound = Path(*[segment for path in paths for segment in path])
    new_file = file[:-4] + suffix + file[-4:]
    wsvg(compound, attributes=attributes,svg_attributes=svg_attributes, filename=new_file)
    return new_file

In [ ]:
'''combine path'''
for i in range(495,496):
    num = str(i)
    if len(num) == 1 :
        num = '00' + num
    if len(num) == 2:
        num = '0' + num
    file = "output/sunflowers_rendered_single_stroke_0" + num + ".svg"
    try:
        new_file = glob(file, '_combine')
        print(file, '->', new_file)
    except Exception as e:
        print('ERROR:', file, e.split('\n')[0])

In [ ]:
import xml.etree.ElementTree as ET

svg = ET.Element('svg', width='512', height='512')


for i in range(1,495):
    group = ET.Element('g')
    num = str(i+1)
    if len(num) == 1 :
        num = '00' + num
    if len(num) == 2:
        num = '0' + num
    file = "output/test/sunflowers_rendered_single_stroke_0" + num + ".svg"
    tree1 = ET.parse(file)
    root1 = tree1.getroot()
    for path in root1.iter('{http://www.w3.org/2000/svg}path'):
        group.append(path)
    print(group)
    svg.append(group)
tree = ET.ElementTree(svg)   
tree.write('output/test/merged.svg')

<Element '{http://www.w3.org/2000/svg}path' at 0x7fb5b8f78d70>
<Element '{http://www.w3.org/2000/svg}path' at 0x7fb5b8f78d10>
<Element '{http://www.w3.org/2000/svg}path' at 0x7fb5b8f78bf0>
<Element '{http://www.w3.org/2000/svg}path' at 0x7fb5b8f78a10>
<Element '{http://www.w3.org/2000/svg}path' at 0x7fb5b8f78b90>
<Element '{http://www.w3.org/2000/svg}path' at 0x7fb5b8f788f0>
<Element '{http://www.w3.org/2000/svg}path' at 0x7fb5b8f78890>
<Element '{http://www.w3.org/2000/svg}path' at 0x7fb5b8f78830>
<Element '{http://www.w3.org/2000/svg}path' at 0x7fb5b8f787d0>
<Element '{http://www.w3.org/2000/svg}path' at 0x7fb5b8f786b0>
<Element '{http://www.w3.org/2000/svg}path' at 0x7fb5b8f785f0>
<Element '{http://www.w3.org/2000/svg}path' at 0x7fb5b8f78590>
<Element '{http://www.w3.org/2000/svg}path' at 0x7fb5b8f78530>
<Element '{http://www.w3.org/2000/svg}path' at 0x7fb5b8f78410>
<Element '{http://www.w3.org/2000/svg}path' at 0x7fb5b8f78230>
<Element '{http://www.w3.org/2000/svg}path' at 0x7fb5b8

In [ ]:
'''move file'''
import os
import shutil
import copy
for i in range(1,495):
    num = str(i)
    if len(num) == 1 :
        num = '00' + num
    if len(num) == 2:
        num = '0' + num
    file = "log/test/sunflowers_rendered_single_stroke_0" + num + "/output-svg/5.svg"
    direct = "output/test/sunflowers_rendered_single_stroke_0" + num + ".svg"
    #current = "log/" + name[0] + "/output-svg/1.svg"
    shutil.move( file, direct )

In [ ]:
import numpy as np
import PIL
import PIL.Image
import torch


use_gpu = torch.cuda.is_available()
device = torch.device('cuda') if use_gpu else torch.device('cpu')

gt = np.array(PIL.Image.open("data/single_strokes/single_strokes_495/sunflowers_rendered_single_stroke_0041.png"))

if gt.shape[2] == 4:
    print("Input image includes alpha channel, simply dropout alpha channel.")
    gt = gt[:, :, :3]

In [2]:

gt = (gt/255).astype(np.float32)
gt = torch.FloatTensor(gt).permute(2, 0, 1)[None].to(device)


In [3]:
h, w = gt.shape[2:]
para_bg = torch.tensor([1., 1., 1.], requires_grad=False, device=device)
pred = para_bg.view(1, -1, 1, 1).repeat(1, 1, h, w)

In [4]:
pred = pred.detach().cpu().numpy()
gt = gt.detach().cpu().numpy()

In [5]:
map = ((pred[0] - gt[0])**2).sum(0)

In [ ]:

#map[map < 0.1]=0
std = np.std(map)
var = np.var(map)

# 輸出結果
print("數據的標準差為：", std)
print("數據的變異數為：", var)

if std < 0.2:
    print("數據比較集中")
else:
    print("數據比較分散")


In [ ]:

q1 = np.percentile(map, 25)
q3 = np.percentile(map, 75)
iqr = q3 - q1

print("Q1:", q1)
print("Q3:", q3)
print("IQR:", iqr)

Q1: 0.0
Q3: 0.0
IQR: 0.0


In [35]:
print(len(map[map==0.0]))

260828


In [6]:
w, h = map.shape
pixel_num = w*h
print(pixel_num)
zero_count = len(map[map==0.0])
print(zero_count)
percent = zero_count / pixel_num
print(percent)

262144
260828
0.9949798583984375


In [7]:
if percent > 0.99 :
    qi = 500

In [16]:
print(map[map!=0.0])

[0.19397153 0.24756628 0.19078813 ... 0.16064587 0.11552478 0.11744712]


In [8]:
map[map < 0.1] = 0


In [9]:
quantile_interval = np.linspace(0., 1., qi)
quantized_interval = np.quantile(map, quantile_interval)

In [10]:
q = np.quantile(map,0.999)
print(q)

0.20661277499794606


In [11]:
quantized_interval = np.unique(quantized_interval)
print(quantized_interval)

[0.         0.14817375 0.18762013 0.6654824 ]


In [12]:
quantized_interval_1 = sorted(quantized_interval)
quantized_interval_2 = sorted(quantized_interval[1:-1])
print(quantized_interval_1)
print(quantized_interval_2)

[0.0, 0.1481737494468689, 0.1876201331615448, 0.6654824018478394]
[0.1481737494468689, 0.1876201331615448]


In [13]:
map = np.digitize(quantized_interval_2, quantized_interval, right=False)

In [14]:
print(map)

[2 3]


In [15]:
map = np.clip(map, 0, 255).astype(np.uint8)
idcnt = {}
for idi in sorted(np.unique(map)):
    idcnt[idi] = (map==idi).sum()
print(idcnt)
idcnt.pop(min(idcnt.keys()))    

{2: 1, 3: 1}


1

In [16]:
target_id = max(idcnt, key=idcnt.get)

In [17]:
print(target_id)

3


In [18]:
import cv2
_, component, cstats, ccenter = cv2.connectedComponentsWithStats(
            (map==target_id).astype(np.uint8), connectivity=4)

In [19]:
csize = [ci[-1] for ci in cstats[1:]]
target_cid = csize.index(max(csize))+1
center = ccenter[target_cid][::-1]
coord = np.stack(np.where(component == target_cid)).T
dist = np.linalg.norm(coord-center, axis=1)
target_coord_id = np.argmin(dist)
coord_h, coord_w = coord[target_coord_id]
# replace_sampling
idcnt[target_id] -= max(csize)
if idcnt[target_id] == 0:
    idcnt.pop(target_id)
print(coord_w, coord_h)

0 1


In [28]:
import copy
radius = 5
center = pos_init_method
color_ref = copy.deepcopy(center)
points = get_bezier_circle(
                radius=radius, segments=4,
                bias=center)
print(points)

tensor([[ 5.0000e+00,  1.0000e+00],
        [ 4.3301e+00,  3.5000e+00],
        [ 2.5000e+00,  5.3301e+00],
        [ 3.0616e-16,  6.0000e+00],
        [-2.5000e+00,  5.3301e+00],
        [-4.3301e+00,  3.5000e+00],
        [-5.0000e+00,  1.0000e+00],
        [-4.3301e+00, -1.5000e+00],
        [-2.5000e+00, -3.3301e+00],
        [-9.1849e-16, -4.0000e+00],
        [ 2.5000e+00, -3.3301e+00],
        [ 4.3301e+00, -1.5000e+00]])


In [ ]:
! python main.py --config config/base.yaml --experiment experiment_1x1 --signature sunflowers --target data/single_strokes/single_strokes_495/sunflowers_rendered_single_stroke_0041.png --log_dir log/test